In [4]:
print('..running')
HYPERPARAMETER_OPTIMIZE = True
import os
import torch
from torch.utils.data import DataLoader
import torch.optim.lr_scheduler as lr_scheduler
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
import ssl
ssl._create_default_https_context = ssl._create_unverified_context
from util import translate_img_batch, cross_entropy_loss_fn, plot_curve
import models
import wandb
from train import evaluation, training 
from data import load_data

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

result_dir = 'models'
if not(os.path.exists(result_dir)):
    os.mkdir(result_dir)
name = 'made' #Change to regularized

batch_size = 136
D = 784   # input dimension
M = 8000  # hidden layer dimensionality
lr = 1e-2 # learning rate
num_epochs = 100
max_patience = 5 # Early Stopping
lam = 1.
n_masks = 1

hyperparameters = {'D': D, 
                   'M': M,
                   'lr': lr,
                   'n_masks': n_masks,
                   'num_epochs': num_epochs,
                   'max_patience': max_patience,
                   'batch_size': batch_size,
                   'lambda': lam,
                    }

run = wandb.init(entity="rajpal906", project="MADE", name="regularized", id="2", config=hyperparameters, settings=wandb.Settings(start_method="fork"))
# Training procedure
train_data, val_data, test_data = load_data('mnist', binarize = True)
train_loader = DataLoader(train_data, batch_size=batch_size, shuffle=True, num_workers=os.cpu_count())
val_loader = DataLoader(val_data, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
test_loader = DataLoader(test_data, batch_size=batch_size, shuffle=False, num_workers=os.cpu_count())
model = models.MADE(input_dim=D, hidden_dims=[M], n_masks=n_masks).to(device)
optimizer = torch.optim.Adam([p for p in model.parameters() if p.requires_grad == True], lr = 1e-3)
scheduler = lr_scheduler.StepLR(optimizer, step_size=100, gamma=0.5)
nll_val = training(name=name, result_dir=result_dir, max_patience=max_patience, num_epochs=num_epochs, model=model, loss_fn=cross_entropy_loss_fn, optimizer=optimizer,
                scheduler=scheduler, training_loader=train_loader, val_loader=val_loader, device=device, lam=lam)
model.eval()
test_val = evaluation(test_loader, cross_entropy_loss_fn, model_best=model)#_best)
wandb.log({"test_loss": test_val * batch_size})
run.log_artifact(result_dir + '/' + name + '.model')
run.finish()

..running


Epoch: 0, train nll=215.60369873046875, val nll=0.8092041231791178
saved!
Epoch: 1, train nll=207.42648315429688, val nll=0.7498119557698568
saved!
Epoch: 2, train nll=228.54071044921875, val nll=0.739297612508138
saved!
Epoch: 3, train nll=198.93482971191406, val nll=0.7199281425476074
saved!
Epoch: 4, train nll=194.0862579345703, val nll=0.7133202679951985
saved!
Epoch: 5, train nll=174.180419921875, val nll=0.7027193450927735
saved!
Epoch: 6, train nll=188.07037353515625, val nll=0.6959056383768717
saved!
Epoch: 7, train nll=189.48187255859375, val nll=0.6942255172729492
saved!
Epoch: 8, train nll=214.17242431640625, val nll=0.6922656529744466
saved!
